# Amazon SageMaker TensorFlow Custom Inference.py Tabular/Regression Example

<b>Notebook Setting</b>
- Kernel: conda_tf2_p36
- Notebook Instance: ml.c5.xlarge

<b>Notebook Scope</b>
- Dataset: Boston Housing Regression
- Model Training with TF ANN script
- Deployment with custom inference.py script for a tabular dataset

## Imports

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import boto3
import sagemaker
import os
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import TensorFlowModel

## Role and S3 Buckets

In [3]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0' #adjust for your version
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'regression-models'

# Boston Housing Model Training

## Boston Dataset Creation

In [4]:
from sklearn import datasets #Boston Housing
boston = datasets.load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target)
y.columns=['TARGET']
df = pd.concat([X,y], axis=1)

#split into train and test to push to local
bostonTrain = df.iloc[:450,:]
bostonTest = df.iloc[451:,:]

In [5]:
DATASET_PATH = './Data/Boston'
os.makedirs(DATASET_PATH, exist_ok=True)

In [6]:
bostonTrain.to_csv('Data/Boston/train.csv', index=False)
bostonTest.to_csv('Data/Boston/test.csv', index=False)

In [7]:
!aws s3 cp ./{DATASET_PATH}/train.csv s3://{BUCKET}/{PREFIX}/BostonHousing/train/

upload: Data/Boston/train.csv to s3://sagemaker-us-east-1-474422712127/regression-models/BostonHousing/train/train.csv


## Create Training Inputs Boston Model

In [8]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/BostonHousing/train',content_type='csv')

In [9]:
inputs = {'train': train_input}

## Boston Model Training

In [10]:
model_name = 'bostonhousing-ann'
hyperparameters = {'epochs': 50}
estimator_parameters = {'source_dir':"Scripts",
                        'entry_point':'boston.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'hyperparameters': hyperparameters,
                        'output_path': f's3://{BUCKET}/{PREFIX}/BostonHousing/out',
                        'base_job_name': f'mme-cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}
estimator_1 = TensorFlow(**estimator_parameters)
estimator_1.fit(inputs)

2021-11-15 23:53:39 Starting - Starting the training job...
2021-11-15 23:54:06 Starting - Launching requested ML instancesProfilerReport-1637020419: InProgress
......
2021-11-15 23:55:06 Starting - Preparing the instances for training.........
2021-11-15 23:56:26 Downloading - Downloading input data
2021-11-15 23:56:26 Training - Downloading the training image..2021-11-15 23:56:48,826 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-11-15 23:56:48,834 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-15 23:56:49,204 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-15 23:56:49,220 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-15 23:56:49,234 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-15 23:56:49,244 sagemaker-training-toolkit INFO     Invoking user script
Trainin

## Create Boston Model

In [11]:
model_1 = estimator_1.create_model(role=role, source_dir="Scripts", entry_point="inference.py")

## Create Boston Endpoint

In [12]:
predictorA = model_1.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## Test Boston Endpoint

In [13]:
predictorA.serializer = sagemaker.serializers.JSONSerializer()
predictorA.deserializer = sagemaker.deserializers.JSONDeserializer()

In [14]:
#Grab sample test data
test = pd.read_csv('Data/Boston/train.csv')
test[:1]
testX = test.drop("TARGET", axis=1)
testX = testX[:1].values.tolist()
sampInput = {"inputs": testX}
sampInput

{'inputs': [[0.00632,
   18.0,
   2.31,
   0.0,
   0.5379999999999999,
   6.575,
   65.2,
   4.09,
   1.0,
   296.0,
   15.3,
   396.9,
   4.98]]}

In [15]:
predicted_value = predictorA.predict(sampInput)
predicted_value

{'outputs': [[35.0728226]]}

## Boto3 Invocation

In [18]:
import boto3
import json
from sagemaker.serializers import JSONSerializer

runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
endpoint_name = predictorA.endpoint_name
jsons = JSONSerializer()
payload = jsons.serialize(sampInput)
response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload)
result = json.loads(response['Body'].read().decode())['outputs']
result

[[35.0728226]]

## Delete Endpoint

In [ ]:
predictorA.delete_endpoint(delete_endpoint_config=True)